# READING POINT CLOUD REFERENCE DATA FOR POSE MAPPING
In this notebook, we evaluate the RDF GRAPHS in a session repository.
For every POINTCLOUDNODE resource, a POINTCLOUDNODE metadata class is created and its data is loaded. 
The following files will be imported and represented in memory
>pcdnodelist=PointCloudNode[]

>This codebase operates on the scan2bim.yml environment (python 3.8)

In [4]:
# import APIs
import rdflib
from rdflib import Graph, plugin
from rdflib.serializer import Serializer #pip install rdflib-jsonld https://pypi.org/project/rdflib-jsonld/
from rdflib import Graph
from rdflib import URIRef, BNode, Literal
from rdflib.namespace import CSVW, DC, DCAT, DCTERMS, DOAP, FOAF, ODRL2, ORG, OWL, \
                           PROF, PROV, RDF, RDFS, SDO, SH, SKOS, SOSA, SSN, TIME, \
                           VOID, XMLNS, XSD

import os.path, time
import importlib
import PIL
import PIL.Image as PILimage
from PIL import ImageDraw, ImageFont, ImageEnhance
from PIL.ExifTags import TAGS, GPSTAGS
import xml.etree.ElementTree as ET
import numpy as np
import open3d as o3d

#IMPORT MODULES
import Algorithms.linkeddatatools as ld
import Classes.pointcloudnode as pc
import Algorithms.scan2bim as s2b

## 1. INITIALIZE SESSION

In [5]:
## INPUTS
project_path= "D:\\Data\\2018-06 Werfopvolging Academiestraat Gent"
session_path = project_path + "\\week 22\\PCD"
bim_folderpath = project_path + "\\BIM"
pcd_folderpath = session_path + "\\PCD"
img_folderpath = session_path + "\\IMG"
ortho_folderpath =session_path + "\\ORTHO"
mesh_folderpath = session_path + "\\MESH"

## 2. READ AND MERGE All THE RDF GRAPHS IN THE SESSION (save for feature.ttl files)

In [15]:
# read all graph files in a session and merge them
importlib.reload(ld)
importlib.reload(pc)
importlib.reload(s2b)
all_session_file_paths=ld.getListOfFiles(session_path) 
rdf_graph_paths=[]
pcdnodelist=[]

# collect all RDF graphs in the session
session_graph = Graph()
for file in all_session_file_paths:        
    if (file.endswith(".ttl") and "features" not in file): 
        rdf_graph_paths.append(file)        
        #session_graph.parse(file) # this becomes 1 big massive graph 
        #print(session_graph.serialize())

session_graph.parse(rdf_graph_paths[0])
print(str(len(rdf_graph_paths))+' RDF Graph files found with '+str(len(session_graph)) +' triples')

1 RDF Graph files found with 912 triples


In [16]:
#test
print(session_graph.serialize())

@prefix e57: <http://libe57.org/> .
@prefix gom: <https://w3id.org/gom#> .
@prefix openlabel: <https://www.asam.net/index.php?eID=dumpFile&t=f&f=3876&token=413e8c85031ae64cc35cf42d0768627514868b2f> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix v4d3D: <https://w3id.org/v4d/core#> .

<http://academiestraat_week_22_1> a "https://w3id.org/v4d/3D#PointCloudNode" ;
    rdfs:label "{fa918228-3168-4f42-9df1-846d3a0324bb}" ;
    e57:cartesianBounds "[-5.417961120605469, 48.5119743347168, 3.067720413208008, 94.35964965820312, 131.4711151123047, 51.0487060546875]" ;
    e57:recordCount "12071241" ;
    gom:hasCoordinateSystem "local" ;
    v4d3D:accuracy "0.05" ;
    v4d3D:e57_path "D:\\Data\\2018-06 Werfopvolging Academiestraat Gent\\week 22\\PCD\\week 22 lidar_CC.e57" ;
    v4d3D:e57_xml_path "D:\\Data\\2018-06 Werfopvolging Academiestraat Gent\\week 22\\PCD\\week 22 lidar_CC.xml" ;
    v4d3D:features3d_path "None" ;
    v4d3D:images2d_path "None" ;
    v4d3D:pcd_path "D:\\D

In [81]:
#report all prefixen (optional)
for prefix, namespace in session_graph.namespaces():
    print(prefix)        

brick
csvw
dc
dcat
dcmitype
dcterms
dcam
doap
foaf
odrl
org
owl
prof
prov
qb
rdf
rdfs
schema
sh
skos
sosa
ssn
time
vann
void
xsd
xml
e57
gom
openlabel
v4d3D
v4d3D1


# 3. A PointCloudNode is created per point cloud subject in the joint RDF Graph

In [19]:
importlib.reload(ld)
importlib.reload(pc)
importlib.reload(s2b)

v4d3D=rdflib.Namespace('https://w3id.org/v4d/3D#')
session_graph.bind('v4d3D', v4d3D)
openlabel=rdflib.Namespace('https://www.asam.net/index.php?eID=dumpFile&t=f&f=3876&token=413e8c85031ae64cc35cf42d0768627514868b2f')
session_graph.bind('openlabel', openlabel)
e57=rdflib.Namespace('http://libe57.org/')
session_graph.bind('e57', e57)

for s in session_graph.subjects(RDF.type, Literal(v4d3D.PointCloudNode)):
    # print(str(session_graph.value(subject=s,predicate=e57.recordCount) ))  
    pcdnode=pc.create_pointcloudnode_from_rdf(session_graph=session_graph,s=s)
    pcdnodelist.append(pcdnode)

print (str(len(pcdnodelist))+' PointCloudNodes succesfully created from the session!')

# alternative search options 
# find all subjects (s) of type (rdf:type) person (foaf:Person)
# for s, p, o in session_graph.triples((None, RDF.type, 'v4d3D.PointCloudNode')): #subjects(s), predicates (p) & objects (o) 
    # print(f"{s} is a PointCloudNode")

144 PointCloudNodes succesfully created from the session!


## 4. IMPORT O3D.POINTCLOUDS
import o3d.geometry.pointclouds for each pointcloudnode if present

In [22]:
counter=0
for idx,pcdnode in enumerate(pcdnodelist):
    if pcdnode.pcd_path is not None:
        if pcdnode.o3d_pointcloud is None:
            pcdnode.o3d_pointcloud=o3d.io.read_point_cloud(pcdnode.pcd_path)
            print(str(pcdnode.name)+' succusfully imported')
        counter +=1
    
    elif pcdnode.get_pcd_path() is True: # Return TRUE and set file path if file is found      
        if pcdnode.o3d_pointcloud is None:
            pcdnode.o3d_pointcloud=o3d.io.read_point_cloud(pcdnode.pcd_path)
            print(str(pcdnode.name)+' succusfully imported')
        counter +=1

print (str(counter)+' of '+str(len(pcdnodelist))+ ' PointCloudNodes have their o3d.geometry.pointclouds imported!')

144 of 144 PointCloudNodes have their o3d.geometry.pointclouds imported!
